In [1]:
from gym_match3.envs import Match3Env
import gym
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import torch
import cv2
from PIL import Image
from gym_match3.envs.levels import LEVELS #  default levels
from gym_match3.envs.levels import Match3Levels, Level
from gym_match3.envs.game import (Board,
                                  RandomBoard,
                                  CustomBoard,
                                  Point,
                                  Cell,
                                  AbstractSearcher,
                                  MatchesSearcher)
from gym_match3.envs.game import OutOfBoardError, ImmovableShapeError
from random import choice
from configparser import ConfigParser, ExtendedInterpolation




parser = ConfigParser(interpolation=ExtendedInterpolation())
parser.read('configure.ini')

width_hight = int(parser.get('gym_environment','board_width_and_hight')) 
n_shapesss = int(parser.get('gym_environment','board_number_of_different_color'))

def Getlevels(WnH,shapes):
    LEVELS = [Level(WnH,WnH,shapes, np.zeros((WnH,WnH)).tolist())]
    return LEVELS

env = Match3Env(levels=Match3Levels(Getlevels(width_hight,n_shapesss)))

class One_hot(gym.ObservationWrapper):
    def __init__(self, env=None):
        gym.ObservationWrapper.__init__(self, env)
        
    def observation(self, observation):
        observation = observation.reshape(width_hight ,width_hight).astype(int)
        grid_onehot = np.zeros(shape=(env.n_shapes+1, width_hight , width_hight ))
        table = {i:i for i in range(-1,n_shapesss)} 
        
        for i in range(width_hight):
            for j in range(width_hight):
                grid_element = observation[i][j]
                grid_onehot[table[grid_element]][i][j]=1
        print(grid_onehot)        
        return grid_onehot

#env = One_hot(env)

available_actions = {v : k for k, v in dict(enumerate(env.get_available_actions())).items()}

for i_episode in range(1): #玩 1次遊戲
     
    observation = env.reset() 
    total_reward = 0
    q  = 0
    while True:
        env.render()
       
        validate_move = env.possible_move   # 一般用 env 的屬性紀錄合法走步
        
        #一開始遊戲初始化時,env的屬性會是 None,以及重新一場遊戲時要再 get一次合法走步
        if validate_move == None or len(validate_move)== 0:  
      
            validate_move = env.get_validate_actions()
        
        #print(validate_move)
        
        validate_list = []       
        for i in validate_move:
            if i in available_actions:
                validate_list.append(available_actions.get(i))   
         

        action = choice(validate_list)
        
        
        observation, reward, done, info = env.step(action)  # step函數會檢查下一個 observation 有沒有合法走步,並回傳 observation,如果沒有合法走步 done == True
        q  = q + 1    
        total_reward = total_reward + reward
        print()
        print('observation:')  
        print(observation)
        print('total_reward: ',total_reward)
        print('step',q)
        #print('the swap of coordinate is: ',list(env.get_available_actions()[action]))

        if done:           
            break

matchs_counter:  3

observation:
[[[1. 1. 3. 4. 1. 4. 4.]
  [2. 1. 2. 1. 0. 2. 4.]
  [2. 3. 3. 0. 0. 1. 3.]
  [3. 0. 4. 4. 1. 3. 1.]
  [3. 3. 2. 1. 3. 2. 2.]
  [4. 2. 4. 3. 4. 0. 2.]
  [2. 0. 1. 0. 1. 2. 1.]]]
total_reward:  3
step 1
matchs_counter:  6

observation:
[[[1. 1. 3. 4. 1. 4. 4.]
  [2. 1. 2. 1. 0. 2. 4.]
  [2. 3. 3. 0. 0. 1. 2.]
  [3. 0. 4. 4. 1. 3. 4.]
  [3. 3. 2. 1. 3. 2. 4.]
  [4. 2. 4. 3. 4. 0. 3.]
  [2. 0. 1. 0. 1. 1. 1.]]]
total_reward:  6
step 2
matchs_counter:  9

observation:
[[[1. 1. 3. 4. 0. 3. 3.]
  [2. 1. 2. 1. 1. 4. 4.]
  [2. 3. 3. 0. 0. 2. 4.]
  [3. 0. 4. 4. 0. 1. 2.]
  [3. 3. 2. 1. 1. 3. 4.]
  [4. 2. 4. 3. 2. 3. 4.]
  [2. 0. 1. 0. 4. 0. 3.]]]
total_reward:  9
step 3
matchs_counter:  12

observation:
[[[1. 4. 3. 4. 0. 3. 3.]
  [2. 3. 2. 1. 1. 4. 4.]
  [2. 3. 3. 0. 0. 2. 4.]
  [0. 1. 4. 4. 0. 1. 2.]
  [3. 1. 2. 1. 1. 3. 4.]
  [4. 2. 4. 3. 2. 3. 4.]
  [2. 0. 1. 0. 4. 0. 3.]]]
total_reward:  12
step 4
matchs_counter:  15

observation:
[[[1. 4. 3. 4. 0. 4. 3.]
  [

KeyboardInterrupt: 

In [4]:
env = Match3Env(levels=Match3Levels(Getlevels(5,5)))



available_actions = {v : k for k, v in dict(enumerate(env.get_available_actions())).items()}


In [18]:
observation = env.reset()
print(observation)
observation, reward, done, info = env.step(action = 1)
print(observation)

[[[3. 2. 3. 0. 4.]
  [4. 1. 3. 1. 4.]
  [3. 4. 0. 2. 1.]
  [1. 4. 3. 4. 1.]
  [1. 2. 1. 0. 4.]]]
matchs_counter:  3
[[[3. 2. 3. 0. 4.]
  [4. 1. 3. 1. 4.]
  [3. 4. 0. 2. 1.]
  [1. 4. 3. 4. 1.]
  [1. 2. 1. 0. 4.]]]


In [1]:
from gym_match3.envs import Match3Env
import gym
import matplotlib.pyplot as plt

from matplotlib import style
import numpy as np

env = Match3Env()
#obs, reward, done, info = env.step(0) 
import torch
import cv2

from PIL import Image

from gym_match3.envs.levels import LEVELS #  default levels
from gym_match3.envs.levels import Match3Levels, Level


from gym_match3.envs.game import (Board,
                                  RandomBoard,
                                  CustomBoard,
                                  Point,
                                  Cell,
                                  AbstractSearcher,
                                  MatchesSearcher)


from gym_match3.envs.game import OutOfBoardError, ImmovableShapeError

def one_hot_encoding(obs):
    
    obs = obs.reshape(obs.shape[1],obs.shape[2]).astype(int)
    
    grid_onehot = np.zeros(shape=(env.n_shapes, obs.shape[0], obs.shape[1]))
    table = {i:i for i in range(-1,10)}
    
    for i in range(obs.shape[0]):
        for j in range(obs.shape[1]):
            grid_element = obs[i][j]
            grid_onehot[table[grid_element]][i][j]=1
            
    return grid_onehot



    
    
def Getlevels(WidthAndHeight,shapes):
    LEVELS = [Level(WidthAndHeight,WidthAndHeight,shapes, np.zeros((WidthAndHeight,WidthAndHeight)).tolist())]
    return LEVELS




In [3]:
from gym_match3.envs import Match3Env
import gym
from PIL import Image
import cv2
import numpy as np
from random import choice


env = Match3Env(levels=Match3Levels(Getlevels(5,5)))



available_actions = {v : k for k, v in dict(enumerate(env.get_available_actions())).items()}



for i_episode in range(1): #玩 1次遊戲
     
    observation = env.reset() 
    
    while True:
        env.render()
       
        validate_move = env.possible_move   # 一般用 env 的屬性紀錄合法走步
        
        
        
        #一開始遊戲初始化時,env的屬性會是 None,以及重新一場遊戲時要再 get一次合法走步
        if validate_move == None or len(validate_move)== 0:  
      
            validate_move = env.get_validate_actions()
        
        #print(validate_move)
        
        validate_list = []       
        for i in validate_move:
            if i in available_actions:
                validate_list.append(available_actions.get(i))   
         

        action = choice(validate_list)
        
        
        observation, reward, done, info = env.step(action)  # step函數會檢查下一個 observation 有沒有合法走步,並回傳 observation,如果沒有合法走步 done == True
            
        
        print('the swap of coordinate is: ',list(env.get_available_actions()[action]))

        if done:           
            break


matchs_counter:  3
the swap of coordinate is:  [(0, 0), (0, 1)]
matchs_counter:  6
the swap of coordinate is:  [(1, 0), (2, 0)]
matchs_counter:  10
the swap of coordinate is:  [(1, 1), (1, 2)]
matchs_counter:  13
the swap of coordinate is:  [(2, 3), (3, 3)]
matchs_counter:  16
the swap of coordinate is:  [(3, 4), (4, 4)]
matchs_counter:  19
the swap of coordinate is:  [(1, 1), (1, 2)]
matchs_counter:  24
the swap of coordinate is:  [(3, 3), (3, 4)]
matchs_counter:  27
the swap of coordinate is:  [(4, 1), (4, 2)]
matchs_counter:  30
the swap of coordinate is:  [(3, 0), (3, 1)]
matchs_counter:  33
the swap of coordinate is:  [(0, 0), (1, 0)]
